# Explore the IMF API
This notebook attempts to dociument my exploration of the API and hopefully help with documenting decisions about the API.

We are going to use [this documentation](http://datahelp.imf.org/knowledgebase/articles/667681-using-json-restful-web-service) from the IMF to explore the db.

In [19]:
import requests
import random
from pprint import pprint
IMF_BASE_API = "http://dataservices.imf.org/REST/SDMX_JSON.svc"

In [8]:
def get_data(url):
    response = requests.get(url)
    response.raise_for_status()
    return response.json()

In [12]:
dataflow_data = get_data(f"{IMF_BASE_API}/Dataflow") # TODO: maybe use the python path lib for urls?
pprint(dataflow_data, depth=1)
print()
pprint(dataflow_data, depth=2)

{'Structure': {...}}

{'Structure': {'@xmlns': 'http://www.SDMX.org/resources/SDMXML/schemas/v2_0/message',
               '@xmlns:xsd': 'http://www.w3.org/2001/XMLSchema',
               '@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance',
               '@xsi:schemaLocation': 'http://www.SDMX.org/resources/SDMXML/schemas/v2_0/message '
                                      'https://registry.sdmx.org/schemas/v2_0/SDMXMessage.xsd',
               'Dataflows': {...},
               'Header': {...}}}


Let's explore the `Header` and `DataFlows`

In [15]:
pprint(dataflow_data["Structure"]["Header"], depth=3)

{'ID': 'abf18cb4-4736-4e0c-bb57-a9a48447e4f7',
 'Prepared': '2020-05-26T07:49:33',
 'Receiver': {'@id': 'ZZZ'},
 'Sender': {'@id': '1C0',
            'Contact': {'Telephone': '+ 1 (202) 623-6220',
                        'URI': 'http://www.imf.org'},
            'Name': {'#text': 'IMF', '@xml:lang': 'en'}},
 'Test': 'false'}


`Header` Just gives us some metadata

In [17]:
pprint(dataflow_data["Structure"]["Dataflows"], depth=2)

{'Dataflow': [{...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
             

Let's have a look at the dimensions of the data

In [20]:
list_of_dataflows = dataflow_data["Structure"]["Dataflows"]["Dataflow"]
print(len(list_of_dataflows))

230


Now let's see if the blobs in this list, which we can reasonably conclude are dataflows, have a similar structure.

In [30]:
for i in range(3):
    random_int = random.randrange(len(list_of_dataflows))
    print(list_of_dataflows[random_int].keys())

dict_keys(['@id', '@version', '@agencyID', '@isFinal', '@xmlns', 'Name', 'KeyFamilyRef'])
dict_keys(['@id', '@version', '@agencyID', '@isFinal', '@xmlns', 'Name', 'KeyFamilyRef'])
dict_keys(['@id', '@version', '@agencyID', '@isFinal', '@xmlns', 'Name', 'KeyFamilyRef'])


We can reasonably conclude that the data are the same initial s